In [1]:
!python -V  

Python 3.9.21


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment('nyc-taxi_example')

<Experiment: artifact_location='mlflow-artifacts:/638544247742963827', creation_time=1748057518120, experiment_id='638544247742963827', last_update_time=1748057518120, lifecycle_stage='active', name='nyc-taxi_example', tags={}>

In [6]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [7]:
df_train = read_dataframe('/workspaces/mlops-zoomcamp/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/workspaces/mlops-zoomcamp/data/green_tripdata_2021-02.parquet')

In [8]:
len(df_train), len(df_val)

(73908, 61921)

In [9]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
import xgboost as xgb

In [13]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597059022164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018006244400600163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)
     
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=30,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/codespace/anaconda3/envs/tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:44:41] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23165
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23596
[8]	validation-rmse:7.99324
[9]	validation-rmse:7.78712
[10]	validation-rmse:7.61025
[11]	validation-rmse:7.45955
[12]	validation-rmse:7.33051
[13]	validation-rmse:7.22100
[14]	validation-rmse:7.12714
[15]	validation-rmse:7.04753
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87114
[19]	validation-rmse:6.82743
[20]	validation-rmse:6.78997
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72997
[23]	validation-rmse:6.70550
[24]	validation-rmse:6.68393
[25]	validation-rmse:6.66423
[26]	validation-rmse:6.64810
[27]	validation-rmse:6.63283
[28]	validation-rmse:6.61926
[29]	validation-rmse:6.60775


/home/codespace/anaconda3/envs/tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:45:17] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/05/24 03:45:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run carefree-fox-802 at: http://127.0.0.1:5000/#/experiments/638544247742963827/runs/ef14510af6b3444d9332e1ee45770ac4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/638544247742963827
